# 빅데이터 분석기사 실기 모의고사 1회
***

## 1번 문제 사용 데이터
- cus_seg_train / cus_seg_test / cus_seg_result


## 1-1. Age 변수에 대해 표준화와 정규화를 각각 실시하고 각 결과에 대해 0.1 이상 0.2 미만인 Age 값의 여성 수를 출력하시오.


- 표준화/정규화 구현 여부 + 데이터 핸들링 + 특정 값 추출

In [156]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [157]:
train = pd.read_csv('cus_seg_train.csv')
test = pd.read_csv('cus_seg_test.csv')

In [158]:
train.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,target
0,Male,No,22,No,Healthcare,1.0,Low,4.0,1
1,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,1
2,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,1
3,Male,Yes,56,No,Artist,0.0,Average,2.0,0
4,Male,No,32,Yes,Healthcare,1.0,Low,3.0,0


In [159]:
s_scl = StandardScaler() # 표준화
m_scl = MinMaxScaler() # 정규화

In [161]:
train[['Age']].head()

,Age
0,22
1,67
2,67
3,56
4,32


- train데이터에 대해서만 스케일링을 하기 때문에 fit_transform을 사용한다.
- scaling을 할 때 단일변수가 대상일 경우 "데이터[[컬럼]]" 형태로 scaling을 해야한다.

In [65]:
train['s_scaled'] = s_scl.fit_transform(train[['Age']])
train['m_scaled'] = m_scl.fit_transform(train[['Age']])

In [66]:
train.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,target,s_scaled,m_scaled
0,Male,No,22,No,Healthcare,1.0,Low,4.0,1,-1.303562,0.056338
1,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,1,1.421748,0.690141
2,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,1,1.421748,0.690141
3,Male,Yes,56,No,Artist,0.0,Average,2.0,0,0.755561,0.535211
4,Male,No,32,Yes,Healthcare,1.0,Low,3.0,0,-0.697937,0.197183


In [67]:
df1 = train[(train['s_scaled'] >= 0.1) & (train['s_scaled'] < 0.2)]

In [68]:
df1[df1.Gender == 'Female'].shape[0]

69

In [69]:
df2 = train[(train['m_scaled'] >= 0.1) & (train['m_scaled'] < 0.2)]

In [70]:
df2[df2.Gender == 'Female'].shape[0]

583

## 1-2. Spending Score별로 Age의 3분위수를 모두 더해 하나의 값으로 출력하시오.

In [71]:
train.Spending_Score.unique()

array(['Low', 'High', 'Average'], dtype=object)

### 방법1

In [128]:
df1 = train[train.Spending_Score == 'Low']
df2 = train[train.Spending_Score == 'High']
df3 = train[train.Spending_Score == 'Average']

In [73]:
q1 = np.percentile(df1.Age, 75)
q2 = np.percentile(df2.Age, 75)
q3 = np.percentile(df3.Age, 75)

In [74]:
q1, q2, q3

(46.0, 72.0, 55.0)

In [75]:
q1 + q2 + q3

173.0

### 방법2

In [76]:
train.groupby('Spending_Score').Age.agg(lambda x : np.percentile(x, 75))

Spending_Score
Average    55
High       72
Low        46
Name: Age, dtype: int64

In [77]:
train.groupby('Spending_Score').Age.agg(lambda x : np.percentile(x, 75)).sum()

173

## 1-3. target 예측 모델링을 진행하시오. 평가지표는 Accuracy로 하되 2개 이상의 모델을 사용하고 submission파일의 target값을 0 혹은 1 값으로 채운 후 'result' 파일명으로 제출하시오.

In [78]:
train.drop(['s_scaled', 'm_scaled'], axis = 1, inplace = True)

In [79]:
train['Gender'] = [1 if g == 'Male' else 0 for g in train['Gender']]
test['Gender'] = [1 if g == 'Male' else 0 for g in test['Gender']]

In [80]:
train.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,target
0,1,No,22,No,Healthcare,1.0,Low,4.0,1
1,0,Yes,67,Yes,Engineer,1.0,Low,1.0,1
2,1,Yes,67,Yes,Lawyer,0.0,High,2.0,1
3,1,Yes,56,No,Artist,0.0,Average,2.0,0
4,1,No,32,Yes,Healthcare,1.0,Low,3.0,0


In [81]:
train['Ever_Married'] = [1 if m == 'Yes' else 0 for m in train['Ever_Married']]
test['Ever_Married'] = [1 if m == 'Yes' else 0 for m in test['Ever_Married']]

In [82]:
train.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,target
0,1,0,22,No,Healthcare,1.0,Low,4.0,1
1,0,1,67,Yes,Engineer,1.0,Low,1.0,1
2,1,1,67,Yes,Lawyer,0.0,High,2.0,1
3,1,1,56,No,Artist,0.0,Average,2.0,0
4,1,0,32,Yes,Healthcare,1.0,Low,3.0,0


In [83]:
train['Graduated'] = [1 if m == 'Yes' else 0 for m in train['Graduated']]
test['Graduated'] = [1 if m == 'Yes' else 0 for m in test['Graduated']]

In [84]:
train.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,target
0,1,0,22,0,Healthcare,1.0,Low,4.0,1
1,0,1,67,1,Engineer,1.0,Low,1.0,1
2,1,1,67,1,Lawyer,0.0,High,2.0,1
3,1,1,56,0,Artist,0.0,Average,2.0,0
4,1,0,32,1,Healthcare,1.0,Low,3.0,0


In [85]:
train.groupby('Spending_Score').target.mean()

Spending_Score
Average    0.358974
High       0.419960
Low        0.577811
Name: target, dtype: float64

In [86]:
train['Spending_Score'] = train['Spending_Score'].map({'Average' : 1, 'High' : 2, 'Low' : 3})
test['Spending_Score'] = test['Spending_Score'].map({'Average' : 1, 'High' : 2, 'Low' : 3})

In [87]:
# train = pd.get_dummies(train)

In [88]:
train.head()

,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,target
0,1,0,22,0,Healthcare,1.0,3,4.0,1
1,0,1,67,1,Engineer,1.0,3,1.0,1
2,1,1,67,1,Lawyer,0.0,2,2.0,1
3,1,1,56,0,Artist,0.0,1,2.0,0
4,1,0,32,1,Healthcare,1.0,3,3.0,0


In [91]:
train.groupby('Profession').target.mean().sort_values()

Profession
Artist           0.353234
Lawyer           0.437376
Entertainment    0.450307
Executive        0.481336
Doctor           0.506734
Engineer         0.510239
Homemaker        0.617978
Marketing        0.678112
Healthcare       0.802571
Name: target, dtype: float64

각 profession별로 target이 1일 확률의 평균을 구한다.

In [92]:
pro_rank = train.groupby('Profession').target.mean().sort_values().reset_index(name = 'rank')

In [93]:
pro_rank

,Profession,rank
0,Artist,0.353234
1,Lawyer,0.437376
2,Entertainment,0.450307
3,Executive,0.481336
4,Doctor,0.506734
5,Engineer,0.510239
6,Homemaker,0.617978
7,Marketing,0.678112
8,Healthcare,0.802571


In [94]:
pro_rank['rank'] = range(1, pro_rank.shape[0] + 1)

각 확률을 sort하여 높은 확률에 큰 숫자를 부여하는 rank를 만든다.

In [95]:
pro_rank

,Profession,rank
0,Artist,1
1,Lawyer,2
2,Entertainment,3
3,Executive,4
4,Doctor,5
5,Engineer,6
6,Homemaker,7
7,Marketing,8
8,Healthcare,9


In [97]:
train = pd.merge(train, pro_rank, how = 'left')
test = pd.merge(test, pro_rank, how = 'left')

train, test 데이터에 rank를 추출한 데이터를 병합한다.

In [100]:
train.drop('Profession', axis = 1, inplace = True)
test.drop('Profession', axis = 1, inplace = True)

랭크 생성으로 사용한 Profession 컬럼을 삭제한다.

In [101]:
train.head()

,Gender,Ever_Married,Age,Graduated,Work_Experience,Spending_Score,Family_Size,target,rank
0,1,0,22,0,1.0,3,4.0,1,9
1,0,1,67,1,1.0,3,1.0,1,6
2,1,1,67,1,0.0,2,2.0,1,2
3,1,1,56,0,0.0,1,2.0,0,1
4,1,0,32,1,1.0,3,3.0,0,9


In [102]:
X = train.drop('target', axis = 1)
y = train.target

target = test[X.columns]

In [109]:
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score

### 1. 전체 데이터 fitting

In [110]:
xgb = XGBClassifier(random_state = 42)

In [111]:
xgb.fit(X, y)

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:31:47] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=42,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [112]:
pred = xgb.predict(target)

In [113]:
pred

array([1, 0, 0, ..., 0, 0, 1])

### 2. Ensemble 방식 -> 학습 및 검증 + 예측을 동시에 수행

- KFold -> k-1개 FOLD로 학습하고 1개FOLD로 검증 -> 모델이 1개 생성
- 10FOLD -> 10개 모델생성(각각 다른 모델)
- 각기 다르게 생성된 모델로 최종 예측데이터의 예측값을 도출
- 10개 생성값을 산술평균, 기하평균 등등 -> 최종 submission 형태 제작

In [117]:
skf = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)

In [118]:
xgb_pred = np.zeros((target.shape[0]))
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    xgb = XGBClassifier(random_state = 42)
    xgb.fit(tr_x, tr_y)
    val_pred = xgb.predict_proba(val_x)[:, 1]
    val_pred = [1 if p >= 0.5 else 0 for p in val_pred]
    val_acc = accuracy_score(val_y, val_pred)
    print(val_acc)
    
    fold_pred = xgb.predict_proba(target)[:, 1] / 10
    xgb_pred += fold_pred

/opt/anaconda3/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:36:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.6339285714285714
[21:36:36] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.6145833333333334
[21:36:37] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
0.5967261904761905
[21:36:37] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1095: Starting in XGBoost 1.3

In [119]:
xgb_pred

array([0.57950364, 0.32129632, 0.05599676, ..., 0.42231752, 0.36012891,
       0.68146688])

In [120]:
lgbm_pred = np.zeros((target.shape[0]))
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm = LGBMClassifier(random_state = 42)
    lgbm.fit(tr_x, tr_y)
    val_pred = lgbm.predict_proba(val_x)[:, 1]
    val_pred = [1 if p >= 0.5 else 0 for p in val_pred]
    val_acc = accuracy_score(val_y, val_pred)
    print(val_acc)
    
    fold_pred = lgbm.predict_proba(target)[:, 1] / 10
    lgbm_pred += fold_pred

0.6279761904761905
0.6130952380952381
0.6026785714285714
0.6547619047619048
0.59375
0.6309523809523809
0.6502976190476191
0.6130952380952381
0.669150521609538
0.6110283159463488


In [165]:
submission = pd.read_csv('cus_seg_result.csv')

In [166]:
submission.target = (xgb_pred + lgbm_pred) / 2

In [167]:
submission.head()

,ID,target
0,458989.0,0.560092
1,458994.0,0.382250
2,459000.0,0.173257
3,459003.0,0.305460
4,459005.0,0.638615


In [168]:
submission.target = [1 if p >= 0.5 else 0 for p in submission['target']]

문제에서 0 혹은 1로 저장하라고 명시됐기 때문에 list comprehension으로 0 혹은 1 값으로 변환한다.

In [169]:
submission.head()

,ID,target
0,458989.0,1
1,458994.0,0
2,459000.0,0
3,459003.0,0
4,459005.0,1


In [138]:
#submission.to_csv('result.csv', index = False)

***

## 2번 문제 사용 데이터
- Training Data / Test Data / Sample Prediction Dataset

## 2-1. Married/Single 여부에 따른 CURRENT_HOUSE_YRS 평균값 차이의 절댓값을 출력하시오.
- Married or Single 필터링 가능 여부 + 특정 컬럼에 대한 통계함수 적용 가능 여부 + 최종 요구 값으로 변환 가능 여부

In [183]:
train = pd.read_csv('Training Data.csv')
test = pd.read_csv('Test Data.csv')

### 방법1 - 수작업

In [143]:
m1 = train[train['Married/Single'] == 'single'].CURRENT_HOUSE_YRS.mean()
m2 = train[train['Married/Single'] == 'married'].CURRENT_HOUSE_YRS.mean()

In [145]:
m1, m2

(11.994237024466129, 12.029073383084578)

In [146]:
m2 - m1

0.034836358618449026

In [147]:
abs(m1 - m2)

0.034836358618449026

### 방법2 - 잔꾀

In [149]:
df = train.groupby('Married/Single').CURRENT_HOUSE_YRS.mean()

In [150]:
df

Married/Single
married    12.029073
single     11.994237
Name: CURRENT_HOUSE_YRS, dtype: float64

In [151]:
m = df[0] # married
s = df[1] # single

0.034836358618449026

In [153]:
s, m

(11.994237024466129, 12.029073383084578)

In [154]:
m - s

0.034836358618449026

In [155]:
abs(s - m)

0.034836358618449026

## 2-2. Risk_Flag를 예측하는 모델을 2개 이상 만들어 Precision을 확인하고 test 데이터를 예측하여 submission을 제출하시오. 이 때 제출하는 파일명은 result이고 0 혹은 1값으로 제출해주세요.

In [184]:
train.drop(['Id', 'Profession', 'CITY', 'STATE'], axis = 1, inplace = True)
test.drop(['ID', 'Profession', 'CITY', 'STATE'], axis = 1, inplace = True)

In [187]:
train['Married/Single'] = [1 if m == 'married' else 0 for m in train['Married/Single']]
test['Married/Single'] = [1 if m == 'married' else 0 for m in test['Married/Single']]

In [191]:
train.groupby('House_Ownership').Risk_Flag.mean()

House_Ownership
norent_noown    0.099527
owned           0.089797
rented          0.125577
Name: Risk_Flag, dtype: float64

In [192]:
train['House_Ownership'] = train['House_Ownership'].map({'rented' : 3, 'norent_noown' : 2, 'owned' : 1})
test['House_Ownership'] = test['House_Ownership'].map({'rented' : 3, 'norent_noown' : 2, 'owned' : 1})

In [198]:
train['Car_Ownership'] = [1 if c == 'yes' else 0 for c in train['Car_Ownership']]
test['Car_Ownership'] = [1 if c == 'yes' else 0 for c in test['Car_Ownership']]

In [200]:
X = train.drop('Risk_Flag', axis = 1)
y = train.Risk_Flag

target = test[X.columns]

In [202]:
from sklearn.preprocessing import StandardScaler

In [203]:
scaler = StandardScaler()

In [207]:
X = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)

In [209]:
target = pd.DataFrame(scaler.transform(target), columns = target.columns)

In [211]:
X.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS
0,-1.283145,-1.579604,-1.180232,-0.337200,0.281684,-0.657129,-0.914131,0.716356
1,0.895457,-0.583344,-0.014067,-0.337200,0.281684,-0.657129,0.731036,0.716356
2,-0.349269,0.940348,-1.013637,2.965599,0.281684,-0.657129,-0.639936,-1.427981
3,0.437526,-0.524740,-1.346827,-0.337200,0.281684,1.521772,-1.188325,0.001577
4,0.268128,-0.173119,0.152528,-0.337200,0.281684,-0.657129,-0.914131,1.431135


In [212]:
target.head()

,Income,Age,Experience,Married/Single,House_Ownership,Car_Ownership,CURRENT_JOB_YRS,CURRENT_HOUSE_YRS
0,0.832425,0.530123,1.485289,-0.337200,0.281684,-0.657129,-0.639936,0.716356
1,-1.314007,-1.462396,-0.847042,-0.337200,0.281684,-0.657129,-0.365742,-1.427981
2,1.356432,0.002692,0.319123,-0.337200,0.281684,-0.657129,0.731036,1.431135
3,-1.060588,-0.055912,-0.180662,2.965599,0.281684,1.521772,-0.914131,0.001577
4,-1.731466,-1.462396,1.318694,-0.337200,0.281684,1.521772,1.827814,-0.713202


In [217]:
from sklearn.ensemble import RandomForestClassifier
from lightgbm import LGBMClassifier

from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import precision_score

In [218]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.25, random_state = 42, stratify = y)

In [219]:
X_train.shape, X_val.shape

((189000, 8), (63000, 8))

### 방법1 -> 일반적인 접근

In [237]:
rf = RandomForestClassifier(random_state = 42, max_depth = 3, n_estimators = 150)

In [238]:
rf.fit(X_train, y_train)

RandomForestClassifier(max_depth=3, n_estimators=150, random_state=42)

In [248]:
pred = rf.predict(X_val)

In [249]:
X_val['pred'] = pred

<ipython-input-249-03f5193a5935>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_val['pred'] = pred


In [251]:
X_val.pred.unique()

array([0])

In [253]:
X_train['target'] = y_train

<ipython-input-253-be68b596246c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train['target'] = y_train


In [255]:
X_train.target.value_counts()

0    165753
1     23247
Name: target, dtype: int64

In [240]:
score = precision_score(y_val, pred)

/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [241]:
print(f'{rf.__class__.__name__}의 precision = {score}')

RandomForestClassifier의 precision = 0.0


In [243]:
lgbm = LGBMClassifier(random_state = 42, max_depth = 4, learning_rate = 0.15, n_estimators = 200)

In [244]:
lgbm.fit(X, y)

LGBMClassifier(learning_rate=0.15, max_depth=4, n_estimators=200,
               random_state=42)

In [245]:
pred = lgbm.predict(X_val)

In [246]:
score = precision_score(y_val, pred)

In [247]:
print(f'{lgbm.__class__.__name__}의 precision = {score}')

LGBMClassifier의 precision = 0.7596899224806202


### 방법2 -> Ensemble

In [256]:
skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

In [268]:
lgbm_pred = np.zeros(target.shape[0])

In [269]:
lgbm_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [271]:
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm = LGBMClassifier(random_state = 42, max_depth = 4, learning_rate = 0.15, n_estimators = 200)
    lgbm.fit(tr_x, tr_y)
    val_pred = lgbm.predict_proba(val_x)[:, 1]
    val_pred = [1 if p >= 0.5 else 0 for p in val_pred]
    score = precision_score(val_y, val_pred)
    print(score)
    
    fold_pred = lgbm.predict_proba(target)[:, 1] / 5
    lgbm_pred += fold_pred

0.7105263157894737
0.6432160804020101
0.6918238993710691
0.7216494845360825
0.65625


In [272]:
lgbm_pred

array([0.08644801, 0.08594182, 0.09757197, ..., 0.16575346, 0.08210274,
       0.22262696])

In [273]:
submission = pd.read_csv('Sample Prediction Dataset.csv')

In [275]:
submission['risk_flag'] = lgbm_pred

In [277]:
submission['risk_flag'] = [1 if p >= 0.5 else 0 for p in submission['risk_flag']]

In [279]:
#submission.to_csv('result2.csv', index = False)

***
## 3번 문제 사용 데이터
- airline_train, airline_test, airline_submission

## 3-1. Flight Distance의 이상치인 데이터의 Age의 표준편차를 구하시오.

In [280]:
train = pd.read_csv('airline_train.csv')
test = pd.read_csv('airline_test.csv')

In [282]:
q1 = np.percentile(train['Flight Distance'], 25)
q3 = np.percentile(train['Flight Distance'], 75)

In [283]:
iqr = q3 - q1

In [286]:
l_b = q1 - 1.5 * iqr
u_b = q3 + 1.5 * iqr

In [291]:
df = train[(train['Flight Distance'] < l_b) | (train['Flight Distance'] > u_b)]
df.head()

,satisfaction,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,...,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
20,satisfied,Female,Loyal Customer,13,Personal Travel,Eco,6792,0,5,0,...,1,3,4,5,5,1,3,1,3,0.0
39,dissatisfied,Female,Loyal Customer,31,Business travel,Business,4348,3,1,1,...,3,3,1,1,4,4,4,3,0,1.0
58,dissatisfied,Male,Loyal Customer,7,Personal Travel,Business,4747,3,1,3,...,4,2,1,4,3,4,1,4,0,0.0
107,satisfied,Female,Loyal Customer,19,Business travel,Business,5142,5,5,5,...,5,5,1,5,5,1,4,5,0,0.0
132,dissatisfied,Female,Loyal Customer,32,Business travel,Business,5155,2,1,3,...,2,2,2,2,3,1,3,2,54,51.0


In [292]:
df.Age.std()

10.439408113894972

## 3-2. satisfaction 예측 모델을 roc_auc_score를 기준으로 2개 이상 만들고 확률 값을 submission으로 제출하고 이 때 파일명을 result로 하시오.

In [297]:
train['satisfaction'] = [1 if s == 'satisfied' else 0 for s in train.satisfaction]
test['satisfaction'] = [1 if s == 'satisfied' else 0 for s in test.satisfaction]

In [301]:
train['Gender'] = [1 if g == 'Female' else 0 for g in train['Gender']]
test['Gender'] = [1 if g == 'Female' else 0 for g in test['Gender']]

In [304]:
train['Customer Type'] = [1 if t == 'Loyal Customer' else 0 for t in train['Customer Type']]
test['Customer Type'] = [1 if t == 'Loyal Customer' else 0 for t in test['Customer Type']]

In [307]:
train['Type of Travel'] = [1 if t == 'Business travel' else 0 for t in train['Type of Travel']]
test['Type of Travel'] = [1 if t == 'Business travel' else 0 for t in test['Type of Travel']]

In [313]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [315]:
train.drop('Arrival Delay in Minutes', axis = 1, inplace = True)
test.drop('Arrival Delay in Minutes', axis = 1, inplace = True)

In [317]:
X = train.iloc[:, 1:]
y = train.satisfaction

target = test[X.columns]

### 방법1 - Hold Out

In [318]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, stratify = y, random_state = 42)

In [319]:
lgbm = LGBMClassifier(random_state = 42, max_depth = 4)

In [320]:
lgbm.fit(X_train, y_train)

LGBMClassifier(max_depth=4, random_state=42)

In [321]:
pred = lgbm.predict_proba(X_val)[:, 1]

In [323]:
from sklearn.metrics import roc_auc_score

In [324]:
score = roc_auc_score(y_val, pred)

In [325]:
score

0.9875183148785798

### 방법2 - Ensemble

In [326]:
skf = StratifiedKFold(n_splits = 5, random_state = 42, shuffle = True)

In [327]:
lgbm_pred = np.zeros(target.shape[0])

In [338]:
%%time
for tr_idx, val_idx in skf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx] # 학습용 데이터
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx] # 검증용 데이터
    
    ### 학습 및 검증용 모두 train 데이터로부터 생성됨
    
    lgbm = LGBMClassifier(random_state = 42, max_depth = 4)
    lgbm.fit(tr_x, tr_y) # 학습용 데이터만 학습
    
    val_pred = lgbm.predict_proba(val_x)[:, 1]
    # 검증용 독립변수를 예측한 확률
    score = roc_auc_score(val_y, val_pred)
    # ROC AUC를 평가
    print(score)
    
    
    ### 실제 prediction -> target 데이터 예측
    fold_pred = lgbm.predict_proba(target)[:, 1] / 5
    lgbm_pred += fold_pred

0.9877665451933281
0.9878071531084595
0.988036041263388
0.9877876662890807
0.9871772743350397
CPU times: user 11.1 s, sys: 1.85 s, total: 12.9 s
Wall time: 2.06 s


In [337]:
lgbm_pred = np.mean(lgbm_pred)

In [329]:
lgbm_pred

array([0.97147773, 0.99659181, 0.99129075, ..., 0.9876265 , 0.99425338,
       0.03234098])

In [330]:
submission = pd.read_csv('airline_submission.csv')

In [332]:
submission['satisfaction'] = lgbm_pred

In [334]:
#submission.to_csv('result3.csv', index = False)